In [1]:
import json 
import os 
import cv2
from pprint import pprint
import requests
import numpy as np
import random
from tqdm import tqdm
import math
import albumentations as A
from annotations import LabelMe
import base64

/home/hbdesk/labelstudio_convert/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
LABEL_STUDIO_KEY="eaffb8f4719efb17f9b227ed56fee991c4d06a0a"
LABEL_STUDIO_URL="http://192.168.231.52:8080/"

def xywh2xyxy(xywh, original_width, original_height):
    """
    Convert [x, y, w, h] in percentage format to xyxy absolute coordinates.

    Parameters:
    xywh (list): List of [x, y, w, h] as percentages, where x and y represent the top-left corner.
    original_width (int): Original width of the image.
    original_height (int): Original height of the image.

    Returns:
    tuple: (x_min, y_min, x_max, y_max) in absolute coordinates.
    """
    x_percent, y_percent, width_percent, height_percent = xywh

    x_min = (x_percent / 100) * original_width
    y_min = (y_percent / 100) * original_height
    x_max = x_min + (width_percent / 100) * original_width
    y_max = y_min + (height_percent / 100) * original_height

    return x_min, y_min, x_max, y_max


def bgr2_3grey(image: np.ndarray):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.merge([gray_image, gray_image, gray_image])
    return gray_bgr

def convertImg2b64(imgpath):
    with open(imgpath, "rb") as img_file:
        imgstring = base64.b64encode(img_file.read())
    return imgstring

def get_img_from_studio(image_path, save_path="./captioned_images"):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    download_url = f"{LABEL_STUDIO_URL}{image_path}"
    headers = {
        'Authorization': f'Token {LABEL_STUDIO_KEY}'
    }
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = os.path.basename(image_path)
        save_path = os.path.join(save_path, filename)
        with open(save_path, 'wb') as f:
            f.write(response.content)
            return save_path
        print(f"Image successfully downloaded as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
        return None

def export_label_txt(label_dict_list: list, txt_type: str):
    """
    takes a list of annotation pairs that looks like 
    `Image_path`\t`Label`\n
    """
    if txt_type.lower() == "train":
        with open("rec_gt_train.txt", "w", encoding='utf-8') as train_txt:
            for _a in label_dict_list:
                for key, value in _a.items():
                    train_txt.write(f"{str(key)}\t{str(value)}\n")
    elif txt_type.lower() == "eval":
        with open("rec_gt_eval.txt", "w", encoding='utf-8') as eval_txt:
            for _a in label_dict_list:
                for key, value in _a.items():
                    eval_txt.write(f"{key}\t{str(value)}\n")
    else:
        print(f"invalid export type {txt_type}!")

def separate_train_eval(all_anno: list,train_sep: float = 0.8):
    _all_anno = all_anno.copy() # copy so we dont fuck up the original shit homie
    _train_num = math.floor(len(_all_anno) * train_sep) 
    train_set = []
    _sel_count = _train_num
    for _ in range(_train_num):
        random_index = random.randint(0, (_sel_count - 1))
        _sel_count -= 1
        _sel  = _all_anno[random_index]
        train_set.append(_sel)
        # remove from the copied array and use remaining as eval, avoid copying twice
        _all_anno.pop(_all_anno.index(_sel)) 
    return train_set, _all_anno 

def convert2gray(file_path: str):
    for file in os.listdir(file_path):
        img_path = os.path.join(file_path, file)
        img = bgr2_3grey(cv2.imread(img_path))
        cv2.imwrite(img_path, img)

In [3]:
with open("./bbox_tall2.json", "r") as loadjson:
    data = json.loads(loadjson.read())
pprint(data[0])

{'annotations': [{'completed_by': 1,
                  'created_at': '2024-11-07T01:32:20.845676Z',
                  'draft_created_at': '2024-11-07T01:31:34.162268Z',
                  'ground_truth': False,
                  'id': 8993,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 262.752,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 12,
                  'result': [{'from_name': 'label',
                              'id': 'CGdZlN2Ql-',
                              'image_rotation': 0,
                              'origin': 'manual',
                              'original_height': 21,
                              'original_width': 92,
                              'to_name': 'image',
                              'type': 'rectanglelabels',
                           

In [4]:
final_pairs = []
for annotation in tqdm(data):
    # print(len(annotation["annotations"][0]["result"]))
    if len(annotation["annotations"][0]["result"]) > 0:
        try:
            db_fname = annotation["data"]["image"]
            width = annotation["annotations"][0]["result"][0]["original_width"]
            height = annotation["annotations"][0]["result"][0]["original_height"]
            image_name = get_img_from_studio(db_fname, "/home/hbdesk/labelstudio_convert/labelme_export2/")
            base64_img = convertImg2b64(image_name)
            _annotation = LabelMe(width, height, image_name)
            _annotation.label["flags"] = {"":""}
            _annotation.label["imageData"] = base64_img.decode("utf-8")
            for _bbox in annotation["annotations"][0]["result"]:
                try:
                    xywh = [_bbox["value"]["x"], _bbox["value"]["y"], _bbox["value"]["width"], _bbox["value"]["height"]]
                    x1 , y1 , x2, y2  = xywh2xyxy(xywh, width, height)
                    _annotation.add_label(_bbox["value"]["rectanglelabels"][0], [[x1,y1], [x2,y2]], "rectangle")
                except Exception as _:
                    continue
            json_file = os.path.splitext(image_name)[0] + ".json"
            with open(os.path.join("/home/hbdesk/labelstudio_convert/labelme_export2/", json_file), "w") as writejson:
                writejson.write(json.dumps(_annotation.label, indent=2))
        except:
            continue

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1889/1889 [01:37<00:00, 19.32it/s]
